In [6]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from mSDA import msda
from utils.adaptacion import adapt_msda

#otros
import numpy as np
import pandas as pd

tipo = pruebas[1]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

## Pruebas con el dataset Amazon

In [7]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
1000
data


In [8]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


In [9]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


In [10]:
pr = 0.6
n_layers = 1

# Se entrena una sola vez con todos los datos disponibles
print "Adaptando dominios..."
t_adaptar, train_mappings = adapt_msda(X[:, :dims], pr=pr, n_layers=n_layers)
print "Adaptacion realizada en %.3f segundos." % t_adaptar

Adaptando dominios...
Adaptacion realizada en 249.172 segundos.


In [11]:
#diccionario para mantener los dominios adaptados
adapted = {}

for domain in domains:
    print "Adaptando %s" % domain
    X_tr = np.asarray(labeled[domain]['X_tr'][:, :dims].todense())
    X_ts = np.asarray(labeled[domain]['X_ts'][:, :dims].todense())
    
    tr_mappings, tr_reps = msda.mSDA(X_tr[:, :dims], pr, n_layers, train_mappings)
    ts_mappings, ts_reps = msda.mSDA(X_ts[:, :dims], pr, n_layers, train_mappings)
    
    adapted[domain] = {
        'X_tr': tr_reps[-1],
        'X_ts': ts_reps[-1]
    }

Adaptando kitchen
Adaptando dvd
Adaptando electronics
Adaptando books


In [12]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            #svc = get_best_score(X_tr, y_tr)
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            svc_a = SVC(kernel='linear')
            svc_a = svc_a.fit(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1
    
print "Pruebas completadas"

Tarea 1 de 12
Cargando modelo existente.
Tarea 2 de 12
Cargando modelo existente.
Tarea 3 de 12
Cargando modelo existente.
Tarea 4 de 12
Cargando modelo existente.
Tarea 5 de 12
Cargando modelo existente.
Tarea 6 de 12
Cargando modelo existente.
Tarea 7 de 12
Cargando modelo existente.
Tarea 8 de 12
Cargando modelo existente.
Tarea 9 de 12
Cargando modelo existente.
Tarea 10 de 12
Cargando modelo existente.
Tarea 11 de 12
Cargando modelo existente.
Tarea 12 de 12
Cargando modelo existente.
Pruebas completadas


In [13]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,mSDA,k->d,kitchen,dvd,27.998200,37.25,9.251800
1,mSDA,k->e,kitchen,electronics,28.688278,32.25,3.561722
2,mSDA,k->b,kitchen,books,32.241517,41.00,8.758483
3,mSDA,d->k,dvd,kitchen,23.989994,38.00,14.010006
4,mSDA,d->e,dvd,electronics,28.688278,34.75,6.061722
5,mSDA,d->b,dvd,books,32.241517,38.75,6.508483
6,mSDA,e->k,electronics,kitchen,23.989994,32.50,8.510006
7,mSDA,e->d,electronics,dvd,27.998200,41.50,13.501800
8,mSDA,e->b,electronics,books,32.241517,44.75,12.508483
9,mSDA,b->k,books,kitchen,23.989994,39.50,15.510006


In [15]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/msda/amazon.csv
Resultados guardados.
